# Example of how to classify PII data using Atlan SDK and OpenAI SDK

In [1]:

from atlan_client import get_atlan_client

from pyatlan.model.assets import Table, Column
from pyatlan.model.fluent_search import FluentSearch

import logging
logging.basicConfig(level=logging.INFO)

client = get_atlan_client()
postgres_tables = []
# Get all tables available in the postgres database
request = (
    FluentSearch()
    .where(FluentSearch.asset_type(Table))
    .where(FluentSearch.active_assets())
    .where(Table.CONNECTION_NAME.eq("postgres-ct"))
).to_request()
for result in client.asset.search(request):
    postgres_tables.append(result)


## Get each column's metadata

Iterating through all available tables in the Postgres database, we get the column name and description.

In [2]:
def get_postgres_columns(client, table_qualified_name):
    """
    Get the columns of a Postgres table.
    """
    request = (
        FluentSearch()
        .where(FluentSearch.asset_type(Column))
        .where(FluentSearch.active_assets())
        .where(Column.TABLE_QUALIFIED_NAME.eq(table_qualified_name))
        .include_on_results(Column.DATA_TYPE)
    ).to_request()
    columns = []
    for result in client.asset.search(request):  # type: Column
        definition = {
            "column_name": result.name,
            "column_qualified_name": result.qualified_name,
            "column_description": result.description,
            "column_data_type": result.data_type
        }
        columns.append(definition)
    return columns

table_definitions = []
for table in postgres_tables:
    print(f"Getting column metadata for table: {table.name}")
    columns = get_postgres_columns(client, table.qualified_name)
    table_definition = {
        'table_name': table.name,
        'table_description': table.description,
        'columns': columns
    }
    table_definitions.append(table_definition)
    print("- Columns: " + ", ".join([x['column_name'] for x in columns]))


Getting column metadata for table: CUSTOMERS
- Columns: POSTALCODE, ADDRESS, CUSTOMERID, CONTACTNAME, CITY, COUNTRY, CUSTOMERNAME
Getting column metadata for table: ORDERS
- Columns: ORDERDATE, ORDERID, SHIPPERID, EMPLOYEEID, CUSTOMERID
Getting column metadata for table: CATEGORIES
- Columns: DESCRIPTION, CATEGORYNAME, CATEGORYID
Getting column metadata for table: PRODUCTS
- Columns: PRODUCTID, PRODUCTNAME, PRICE, SUPPLIERID, CATEGORYID, UNIT
Getting column metadata for table: SHIPPERS
- Columns: SHIPPERNAME, PHONE, SHIPPERID
Getting column metadata for table: SUPPLIERS
- Columns: CITY, SUPPLIERNAME, ADDRESS, CONTACTNAME, PHONE, POSTALCODE, COUNTRY, SUPPLIERID
Getting column metadata for table: EMPLOYEES
- Columns: NOTES, FIRSTNAME, BIRTHDATE, EMPLOYEEID, LASTNAME
Getting column metadata for table: ORDERDETAILS
- Columns: ORDERDETAILID, ORDERID, QUANTITY, PRODUCTID


# Send metadata to OpenAI for classification

In [3]:
from openai import OpenAI
from pydantic import BaseModel
import json
from config import OPENAPI_API_KEY

ai_client = OpenAI(api_key=OPENAPI_API_KEY)

class ColumnClassification(BaseModel):
    column_name: str
    column_qualified_name: str
    pi_classification_type: str
    pi_sensitivity_level: str
    pi_classification_certainty: str
    pi_classification_reason: str
    
class TableClassification(BaseModel):
    table_name: str
    table_description: str
    columns: list[ColumnClassification]

class TableClassificationResponse(BaseModel):
    classifications: list[TableClassification]

message = """
You are a data privacy expert trained in global data protection regulations including GDPR, CCPA, and HIPAA. Your task is to classify columns for the presence of Personally Identifiable Information (PII), based on the column name, description, and data type.

You must err on the side of caution: if a column is likely to contain PII based on its name or context — even without full certainty — it should be flagged accordingly.

For each column, return an object with the following fields:
1. column_name: The name of the column.
2. column_qualified_name: The full qualified name of the column as provided.
3. pi_classification_type: The type of PII (e.g., "full_name", "birthdate", "address", "email", "phone", "national_id"). If no PII is present, use "not_pi".
4. pi_sensitivity_level:
   - "high" → SSNs, national IDs, full address, phone number, birthdate, financial data
   - "medium" → full names, email, partial address (e.g., postal code)
   - "low" → demographic data (e.g., city, gender) that becomes identifying when combined with other info
   - If not PII, use "not_pi"
5. pi_classification_certainty:
   - "high" → Very likely to be PII
   - "medium" → Possibly PII based on column name or context
   - "low" → Unclear, but potentially PII
   - "not_pi" → Clearly not PII
6. pi_classification_reason: Brief explanation for the classification (e.g., “Contains full name,” “Likely a unique identifier,” or “Not tied to individuals”)

Examples of PII include:
- Names: "firstname", "lastname", "fullname", "contactname", etc.
- Dates of birth: "birthdate", "dob"
- Location: "address", "postalcode", "city"
- Contact: "email", "phone"
- Identifiers: "ssn", "employeeid", "customerid", "userid"

If any combination of columns (like first name + last name or ZIP + birthdate) could reasonably be used to identify a person, flag them individually based on their contribution to PII risk.

Do not underclassify based on assumptions like "name is too generic" — classify based on **column naming conventions and risk**.

Now evaluate the following table definition:
""" + json.dumps(table_definitions)

completion = ai_client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    # temperature=0.2,
    # top_p = 1,
    response_format=TableClassificationResponse,
    messages=[
        {
            "role": "user",
            "content": message
        }
    ]
)

response_text = completion.choices[0].message.content
ai_response = json.loads(response_text)
print(json.dumps(ai_response, indent=2))


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{
  "classifications": [
    {
      "table_name": "CUSTOMERS",
      "table_description": "",
      "columns": [
        {
          "column_name": "POSTALCODE",
          "column_qualified_name": "default/postgres/1746226578/FOOD_BEVERAGE/SALES_ORDERS/CUSTOMERS/POSTALCODE",
          "pi_classification_type": "not_pi",
          "pi_sensitivity_level": "not_pi",
          "pi_classification_certainty": "not_pi",
          "pi_classification_reason": "Not strictly identifying on its own."
        },
        {
          "column_name": "ADDRESS",
          "column_qualified_name": "default/postgres/1746226578/FOOD_BEVERAGE/SALES_ORDERS/CUSTOMERS/ADDRESS",
          "pi_classification_type": "address",
          "pi_sensitivity_level": "high",
          "pi_classification_certainty": "high",
          "pi_classification_reason": "Contains full address information."
        },
        {
          "column_name": "CUSTOMERID",
          "column_qualified_name": "default/postgres/1746226578/

In [4]:
for table in ai_response['classifications']:
    for column in table['columns']:
        if column['pi_classification_type'] != "not_pi":
            # print(f"Tagging {table['table_name']}.{column['column_name']} as PII data. PI type: {column['pi_classification_type']} - {column['pi_classification_reason']}")
            print(column['column_qualified_name'])
            # client.asset.update_atlan_tags(
            #     asset_type=Column,
            #     qualified_name=column['column_qualified_name'],
            #     atlan_tag_names=['PII']
            # )
    break


default/postgres/1746226578/FOOD_BEVERAGE/SALES_ORDERS/CUSTOMERS/ADDRESS
default/postgres/1746226578/FOOD_BEVERAGE/SALES_ORDERS/CUSTOMERS/CONTACTNAME
default/postgres/1746226578/FOOD_BEVERAGE/SALES_ORDERS/CUSTOMERS/CUSTOMERNAME


In [ ]:
client.asset.update_atlan_tags( # 
    asset_type=Column,
    qualified_name="default/postgres/1746226578/FOOD_BEVERAGE/SALES_ORDERS/CUSTOMERS/ADDRESS",
    atlan_tag_names=["Sensitive Data"]
)


AtlanError: ATLAN-PYTHON-204-000  Suggestion: Check the details of the server's message to correct your request.